<a href="https://colab.research.google.com/github/wangtzi/Pytorch/blob/main/Pytorch_cancer_FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 建立一個名為Cancer的資料夾
! mkdir Cancer

# Download kaggle data in Google Colab
! pip install -q kaggle
from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d bhaveshmittal/melanoma-cancer-dataset
# 將剛剛載下來的.zip壓縮檔解壓縮進Cancer資料夾裡
! unzip melanoma-cancer-dataset.zip -d Cancer

mkdir: cannot create directory ‘Cancer’: File exists


串流輸出內容已截斷至最後 5000 行。
  inflating: Cancer/train/Malignant/1530.jpg  
  inflating: Cancer/train/Malignant/1531.jpg  
  inflating: Cancer/train/Malignant/1532.jpg  
  inflating: Cancer/train/Malignant/1533.jpg  
  inflating: Cancer/train/Malignant/1534.jpg  
  inflating: Cancer/train/Malignant/1535.jpg  
  inflating: Cancer/train/Malignant/1536.jpg  
  inflating: Cancer/train/Malignant/1537.jpg  
  inflating: Cancer/train/Malignant/1538.jpg  
  inflating: Cancer/train/Malignant/1539.jpg  
  inflating: Cancer/train/Malignant/154.jpg  
  inflating: Cancer/train/Malignant/1540.jpg  
  inflating: Cancer/train/Malignant/1541.jpg  
  inflating: Cancer/train/Malignant/1542.jpg  
  inflating: Cancer/train/Malignant/1543.jpg  
  inflating: Cancer/train/Malignant/1544.jpg  
  inflating: Cancer/train/Malignant/1545.jpg  
  inflating: Cancer/train/Malignant/1546.jpg  
  inflating: Cancer/train/Malignant/1547.jpg  
  inflating: Cancer/train/Malignant/1548.jpg  
  inflating: Cancer/train/Malignant/1549

In [7]:
import os
import torch
from pathlib import Path
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets, transforms
from torchvision.datasets import MNIST
from sklearn.metrics import accuracy_score, confusion_matrix , precision_score, recall_score, f1_score, roc_auc_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

path = Path('/content/Cancer/train')
testPath = Path('/content/Cancer/test')

# transforms
transforms_train = transforms.Compose([transforms.RandomAffine(degrees=30),
                    transforms.RandomResizedCrop(size = (200,200),scale=(0.2, 1.0), ratio=(0.5, 1.1)),
                    transforms.CenterCrop(size = (112, 112)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5] , std=[0.5, 0.5, 0.5])])

transforms_test = transforms.Compose([transforms.CenterCrop(size = (112, 112)),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5] , std=[0.5, 0.5, 0.5])])

# data loader
trainData = datasets.ImageFolder(root = path , transform = transforms_train)
testData = datasets.ImageFolder(root = testPath , transform = transforms_test)
batchSize = 64
train_DL = DataLoader(trainData , batch_size = batchSize , shuffle=True)
test_DL = DataLoader(testData , batch_size = batchSize , shuffle=False)

# MODEL
class NN(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*112*112, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )
  def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NN()


# Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train , Optimize
def train_loop(dataloader , model , loss_fn , optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (x,y) in enumerate(dataloader):
    pred = model(x)
    loss = loss_fn(pred, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (batch+1) % 60 ==0:
      loss, current = loss.item(), batch * batch_size + len(x)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# Test
def test_loop(dataloader , model , loss_fn):
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss , correct = 0 , 0
  with torch.no_grad():
    for x,y in dataloader:
      pred = model(x)
      loss = loss_fn(pred, y)
      test_loss += loss
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


# GO
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_DL, model, loss_fn, optimizer)
    test_loop(test_DL, model, loss_fn)
print("Done!")



Using device : cuda
Epoch 1
-------------------------------
loss: 0.651993  [ 3840/11879]
loss: 0.659362  [ 7680/11879]
loss: 0.606330  [11520/11879]
Test Error: 
 Accuracy: 57.2%, Avg loss: 0.614149 

Epoch 2
-------------------------------
loss: 0.643418  [ 3840/11879]
loss: 0.564427  [ 7680/11879]
loss: 0.520782  [11520/11879]
Test Error: 
 Accuracy: 57.6%, Avg loss: 0.610919 

Epoch 3
-------------------------------
loss: 0.620455  [ 3840/11879]
loss: 0.635494  [ 7680/11879]
loss: 0.605737  [11520/11879]
Test Error: 
 Accuracy: 59.2%, Avg loss: 0.599558 

Epoch 4
-------------------------------
loss: 0.604446  [ 3840/11879]
loss: 0.560961  [ 7680/11879]
loss: 0.478863  [11520/11879]
Test Error: 
 Accuracy: 59.2%, Avg loss: 0.598121 

Epoch 5
-------------------------------
loss: 0.508813  [ 3840/11879]
loss: 0.581355  [ 7680/11879]
loss: 0.584778  [11520/11879]
Test Error: 
 Accuracy: 59.5%, Avg loss: 0.593348 

Done!
